# Get all crypto currencies from Binance live
## and save in a sql database

In [1]:
# !pip install python-binance -U


In [2]:
# import libraries
from IPython.display import display
import pandas as pd
import numpy as np
from binance.client import Client
import asyncio
from binance import AsyncClient, BinanceSocketManager
import nest_asyncio
nest_asyncio.apply()
import datetime as dt
from sqlalchemy import inspect
from sqlalchemy import create_engine

In [3]:
engine = create_engine('sqlite:///CryptoDB.db')
client = Client()
info = client.get_exchange_info()
symbols = [s['symbol'] for s in info['symbols']]
exclude = ['UP', 'DOWN', 'BEAR', 'BULL']    
non_lev = [s for s in symbols if all(excludes not in s for excludes in exclude)]
# ends with 'USDT'
relevant = [s for s in non_lev if s.endswith('USDT')]

In [4]:
multi = [i.lower() + '@trade' for i in relevant]

In [5]:
def createframe(msg):
    # display(msg)
    df = pd.DataFrame([msg['data']])
    df = df.loc[:,['s','E','p']]
    df.columns=['symbol', 'Time', 'Price']
    df.Price = df.Price.astype(float)
    df.Time = pd.to_datetime(df.Time, unit='ms')
    return df

# remove from all tables rows with time older than 2 hours

def remove_old_data():
    this_time = dt.datetime.now()
    # add 2 hour to start_time as time limit
    time_limit = this_time - dt.timedelta(hours=2)
    print(f'remove all rows in databse before {time_limit}')
    # loop through all tables in CryptoDB   
    inspector = inspect(engine)
    schemas = inspector.get_schema_names()

    for schema in schemas:
        for table_name in inspector.get_table_names(schema=schema):
            # create df from table_name
            try:
                df = pd.read_sql(f'SELECT * FROM {table_name}', engine)
                df.Time = pd.to_datetime(df.Time, unit='ms')
            except:
                continue    
            
            # delete rows
            df = df[df.Time > time_limit]
            if len(df) > 0:
                # overwritevtable_name with df
                df.to_sql(table_name, engine, if_exists='replace', index=False)        

In [6]:
async def main():
    client = await AsyncClient.create()
    bm = BinanceSocketManager(client)
    ms = bm.multiplex_socket(multi)
    from_time = dt.datetime.now()
    async with ms as tscm:
        while True:
            res = await tscm.recv()
            if res:
                frame = createframe(res)
                frame.to_sql(frame.symbol[0], engine, if_exists='append', index=False)

            elapsed = dt.datetime.now() - from_time
            if elapsed > dt.timedelta(hours=3):
                print('remove old data')
                remove_old_data()
                from_time = dt.datetime.now()
                
                
    await client.close_connection()
                

In [7]:
if __name__ == '__main__':
    loop = asyncio.get_event_loop()
    loop.run_until_complete(main())

remove old data
remove all rows in databse before 2021-11-18 17:50:11.384586
remove old data
remove all rows in databse before 2021-11-18 20:50:14.639717
